```
    def __sub__(self, other):
        '''
        @other: un Tensor
        '''
        if (self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self, other],
                          creation_op='sub')
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        '''
        @other: un Tensor
        '''
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)
    
    def sum(self, dim):
        '''
        Suma atravez de dimensiones, si tenemos una matriz 2x3 y 
        aplicamos sum(0) sumara todos los valores de las filas 
        dando como resultado un vector 1x3, en cambio si se aplica
        sum(1) el resultado es un vector 2x1
        
        @dim: dimension para la suma
        '''
        if(self.autograd):
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim, copies):
        '''
        Se utiliza para retropropagar a traves de una suma sum().
        Copia datos a lo largo de una dimension
        '''

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)
        
        if(self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))

```

In [1]:
import numpy as np


class Tensor(object):

    def __init__(self, data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        '''
        Inicializa un tensor utilizando numpy

        @data: una lista de numeros
        @creators: lista de tensores que participarion en la creacion de un nuevo tensor
        @creators_op: la operacion utilizada para combinar los tensores en el nuevo tensor
        '''
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None
        self.autograd = autograd
        self.children = {}
        # se asigna un id al tensor
        if (id is None):
            id = np.random.randint(0, 100000)
        self.id = id

        # se hace un seguimiento de cuantos hijos tiene un tensor
        # si los creadores no es none
        if (creators is not None):
            # para cada tensor padre
            for c in creators:
                # se verifica si el tensor padre posee el id del tensor hijo
                # en caso de no estar, agrega el id del tensor hijo al tensor padre
                if (self.id not in c.children):
                    c.children[self.id] = 1
                # si el tensor ya se encuentra entre los hijos del padre
                # y vuelve a aparece, se incrementa en uno
                # la cantidad de apariciones del tensor hijo
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        '''
        Verifica si un tensor ha recibido la cantidad
        correcta de gradientes por cada uno de sus hijos
        '''
        # print('tensor id:', self.id)
        for id, cnt in self.children.items():
            if (cnt != 0):
                return False
        return True

    def backward(self, grad, grad_origin=None):
        '''
        Funcion que propaga recursivamente el gradiente a los creators o padres del tensor

        @grad: gradiente
        @grad_orign
        '''
        #         tab=tab
        if (self.autograd):
            if (grad_origin is not None):
                # Verifica para asegurar si se puede hacer retropropagacion
                if (self.children[grad_origin.id] == 0):
                    raise Exception("No se puede retropropagar mas de una vez")
                # o si se está esperando un gradiente, en dicho caso se decrementa
                else:
                    # el contador para ese hijo
                    self.children[grad_origin.id] -= 1

        # acumula el gradiente de multiples hijos
        if (self.grad is None):
            self.grad = grad
        else:
            self.grad += grad

        if (self.creators is not None and
                (self.all_children_grads_accounted_for() or grad_origin is None)):

            if (self.creation_op == 'neg'):
                self.creators[0].backward(self.grad.__neg__())
                
            if (self.creation_op == 'add'):
                # al recibir self.grad, empieza a realizar backprop
                self.creators[0].backward(self.grad, grad_origin=self)
                self.creators[1].backward(self.grad, grad_origin=self)
                
            if(self.creation_op == "sub"):
                self.creators[0].backward(Tensor(self.grad.data), self)
                self.creators[1].backward(Tensor(self.grad.__neg__().data), self)

            if(self.creation_op == "mul"):
                new = self.grad * self.creators[1]
                self.creators[0].backward(new , self)
                new = self.grad * self.creators[0]
                self.creators[1].backward(new, self)                    

            if(self.creation_op == "mm"):
                layer = self.creators[0]                    # activaciones => layer
                weights = self.creators[1]                  # pesos = weights
                # c0 = self.creators[0]                       # activaciones => layer
                # c1 = self.creators[1]                       # pesos = weights
                # new = self.grad.mm(c1.transpose())  # grad = delta => delta x weights.T
                new = Tensor.mm(self.grad, weights.transpose())  # grad = delta => delta x weights.T
                layer.backward(new)
                # c0.backward(new)                            
                # new = self.grad.transpose().mm(c0).transpose() # (delta.T x layer).T = layer.T x delta
                new = Tensor.mm(layer.transpose(), self.grad)  # layer.T x delta
                weights.backward(new)
                # c1.backward(new)

            if(self.creation_op == "transpose"):
                self.creators[0].backward(self.grad.transpose())

            if("sum" in self.creation_op):
                dim = int(self.creation_op.split("_")[1])
                self.creators[0].backward(self.grad.expand(dim, self.creators[0].data.shape[dim]))

            if("expand" in self.creation_op):
                dim = int(self.creation_op.split("_")[1])
                self.creators[0].backward(self.grad.sum(dim))
                
                
    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                         autograd=True,
                         creators=[self],
                         creation_op='neg')
        return Tensor(self.data * -1)
    
    def __add__(self, other):
        '''
        @other: un Tensor
        '''
        if (self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self, other],
                          creation_op='add')
        return Tensor(self.data + other.data)
    
    def __sub__(self, other):
        '''
        @other: un Tensor
        '''
        if (self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self, other],
                          creation_op='sub')
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        '''
        @other: un Tensor
        '''
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)
    
    def sum(self, dim):
        '''
        Suma atravez de dimensiones, si tenemos una matriz 2x3 y 
        aplicamos sum(0) sumara todos los valores de las filas 
        dando como resultado un vector 1x3, en cambio si se aplica
        sum(1) el resultado es un vector 2x1
        
        @dim: dimension para la suma
        '''
        if(self.autograd):
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim, copies):
        '''
        Se utiliza para retropropagar a traves de una suma sum().
        Copia datos a lo largo de una dimension
        '''

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)
        
        if(self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))
    

    def __repr__(self):
        return str(self.data.__repr__())

    def __str__(self):
        return str(self.data.__str__())


```
def __sub__(self, other):
'''
@other: un Tensor
'''
if (self.autograd and other.autograd):
    return Tensor(self.data - other.data,
                  autograd=True,
                  creators=[self, other],
                  creation_op='sub')
return Tensor(self.data - other.data)
```

In [2]:
def zero_grads(tensor: Tensor):
    for c in tensor.creators:
        c.grad.data *= 0

## Resta o ```sub```
Calculamos la resta entre los tensores x1 y x2

In [3]:
x1 = Tensor([2,2,2], autograd=True)
x2 = Tensor([1,1,1], autograd=True)
y = x1-x2
y

array([1, 1, 1])

Ahora queremos calcular el gradiente que deberá ser pasado tanto a x1 como a x2 a partir de esta operacion. Sabemos que:
- x1 es el ```minuendo```
- x2 es el ```sustraendo```
Esto se puede verificar facilmente mirando quienes fueron los creadores de ```y```

In [4]:
print(y.creators[0].id == x1.id)
print(y.creators[1].id == x2.id)
print(y.grad)

True
True
None


En el caso de una resta, el gradiente que debemos pasar a los creadores del tensor y debe ser:
- Para el minuendo, el gradiente del hijo
- Para el sustraendo, el la negacion ( todo por -1) del gradiente del hijo

In [5]:
y_grad = Tensor(np.ones_like(y.data))
x1.grad = y_grad
x2.grad = y_grad.__neg__()
print('x1:',x1.id,'\n', x1.grad)
print('x2: ',x2.id,'\n', x2.grad)
zero_grads(y)

x1: 88906 
 [1 1 1]
x2:  42744 
 [-1 -1 -1]


Implementando esto dentro de la funcion ```backward()```

In [6]:
y.backward(Tensor(np.ones_like(y.data)))

Verificamos que esto se aplique correctamente

In [7]:
print('y.creators[0]: ',y.creators[0].id,'\n', y.creators[0].grad)
print('y.creators[1]: ',y.creators[1].id,'\n', y.creators[1].grad)

y.creators[0]:  88906 
 [1 1 1]
y.creators[1]:  42744 
 [-1 -1 -1]
